# Ноутбук по отбору признаков

[Пример ноутбука в котором идет речь про отбор признаков](https://github.com/RomanSafronenkov/mlcourse.ai/blob/main/jupyter_russian/topic06_features/topic6_feature_engineering_feature_selection_russian.ipynb)

[Sequential Feature Selection с объяснениями](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.feature_selection/#sequentialfeatureselector)

[О том, почему плох встроенный в RandomForest feature_importance](https://explained.ai/rf-importance/)

In [186]:
from abc import ABC, abstractmethod

import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.inspection import permutation_importance

from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt
%matplotlib inline

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
# создадим искусственный датасет, в котором будут информативные признаки, их дубликаты, их комбинации и шумовые признаки
x, y = make_classification(
    n_samples=10000, n_features=100, n_informative=15, n_redundant=5, n_repeated=5, n_clusters_per_class=4, shift=0.8, scale=3.0, shuffle=False)

# если не ставить параметр shuffle в True, то сначала будут идти информативные признаки, потом комбинации и потом повторы, после них - мусор
# так удобнее оценить как это все работает

In [45]:
# замерим метрику на кросс валидации
cross_val_score(LGBMClassifier(verbose=-100), x, y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9491041689783689

In [147]:
class BaseFeatureSelector(ABC):
    def __init__(self, n_folds=5):
        self.n_folds = n_folds
        self.importances = None

    def fit(self, x, y, **importances_kwargs):
        skf = StratifiedKFold(n_splits=self.n_folds, shuffle=True)  # можно число фолдов и прочие параметры

        self.importances = pd.DataFrame({'feature': np.arange(x.shape[1])})

        for i, (train_index, val_index) in enumerate(skf.split(x, y)):
            x_train, y_train = x[train_index], y[train_index]
            x_val, y_val = x[val_index], y[val_index]

            model = LGBMClassifier(max_depth=5, n_estimators=500, learning_rate=0.05, verbose=-100)
            model.fit(x_train, y_train)

            imp = self._get_importances_from_model(model, x_val, y_val, **importances_kwargs)

            self.importances[f'importance_{i}'] = imp

    def get_selected_features(self, threshold):
        assert self.importances is not None, 'Сначала нужно обучить, вызвав метод fit'

        # сделаем отдельно для 0 итерации
        imps = self.importances.loc[:, ['feature', 'importance_0']].sort_values('importance_0', ascending=False)  # выберем важности признаков с 0 итерации
        imps['importance_0'] /= imps['importance_0'].sum()
        imps['cumsum'] = imps['importance_0'].cumsum()  # так как мы их отнормировали, может посчитать кумулятивную сумму
        features = imps.loc[imps['cumsum'] <= threshold, 'feature'].tolist()  # возьмем только те признаки, которые по кумулятивной сумме удовлетворяют
        
        best_features = set(features)  # сделаем множество
        for i in range(self.n_folds):
            imps = self.importances.loc[:, ['feature', f'importance_{i}']].sort_values(f'importance_{i}', ascending=False)
            imps[f'importance_{i}'] /= imps[f'importance_{i}'].sum()
            imps['cumsum'] = imps[f'importance_{i}'].cumsum()
            features = imps.loc[imps['cumsum'] <= threshold, 'feature'].tolist()

            best_features &= set(features)  # смотрим на пересечения множеств на разных итерациях кросс-валидации

        return list(best_features)
        
    @abstractmethod
    def _get_importances_from_model(self, model, x, y, **kwargs):
        pass

In [150]:
# отбор признаков с использованием встроеного feature_importance
class LGMFeatureSelection(BaseFeatureSelector):
    def __init__(self, n_folds=5):
        super().__init__(n_folds)

    def _get_importances_from_model(self, model, x, y, importance_type='split'):
        return model.booster_.feature_importance(importance_type=importance_type)

In [151]:
%%time
lgm_fs = LGMFeatureSelection()
lgm_fs.fit(x, y, importance_type='split')

CPU times: total: 26.1 s
Wall time: 4.37 s


In [152]:
lgm_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,494,481,504,529,483
1,1,251,283,265,292,274
2,2,274,325,259,317,296
3,3,458,481,466,436,548
4,4,399,413,382,420,415
5,5,318,369,324,370,295
6,6,342,375,328,343,312
7,7,336,334,318,337,333
8,8,398,380,340,376,364
9,9,322,324,309,333,302


In [153]:
best_features_lgm_fs = lgm_fs.get_selected_features(threshold=0.7)
len(best_features_lgm_fs)

20

In [154]:
best_features_lgm_fs  # видно, что идут по порядку

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [155]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_lgm_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9609643760995761

In [158]:
# попробуем тоже самое, но с помощью shap values
class ShapFeatureSelection(BaseFeatureSelector):
    def __init__(self, n_folds=5):
        super().__init__(n_folds)

    def _get_importances_from_model(self, model, x, y, feature_perturbation='tree_path_dependent'):
        explainer = shap.TreeExplainer(model, feature_perturbation=feature_perturbation)
        shap_values = explainer.shap_values(x)

        if isinstance(shap_values, list):
            importances = np.abs(shap_values[1]).mean(0)
        else:
            importances = np.abs(shap_values).mean(0)
        return importances

In [159]:
%%time
shap_fs = ShapFeatureSelection()
shap_fs.fit(x, y, feature_perturbation='tree_path_dependent')

CPU times: total: 58.7 s
Wall time: 8.78 s


In [160]:
shap_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,0.245900,0.250060,0.247857,0.266522,0.240047
1,1,0.098704,0.085731,0.086442,0.077808,0.096485
2,2,0.276469,0.255460,0.232303,0.268888,0.246585
3,3,0.435923,0.398038,0.441741,0.407917,0.435214
4,4,0.552919,0.582388,0.567416,0.543746,0.574057
5,5,0.083296,0.091279,0.070517,0.075708,0.081022
6,6,0.104519,0.100954,0.098028,0.104270,0.111093
7,7,0.786311,0.724123,0.734011,0.720951,0.710266
8,8,0.164116,0.184706,0.171061,0.150240,0.162141
9,9,0.202259,0.208244,0.234636,0.203320,0.213477


In [163]:
best_features_shap_fs = shap_fs.get_selected_features(threshold=0.85)
len(best_features_shap_fs)

22

In [164]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_shap_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9589849753321753

In [165]:
# самый интуитивно понятный метод, но самый долгий - permutation importance
class PIFeatureSelection(BaseFeatureSelector):
    def __init__(self, n_folds=5):
        super().__init__(n_folds)

    def _get_importances_from_model(self, model, x, y, **permutation_kwargs):
        importances = np.abs(permutation_importance(model, x, y, **permutation_kwargs)['importances_mean'])
        return importances

In [166]:
%%time
pi_fs = PIFeatureSelection()
pi_fs.fit(x, y, scoring='roc_auc', n_jobs=-1, n_repeats=5)

CPU times: total: 1min 21s
Wall time: 31.9 s


In [167]:
pi_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,0.016977,0.023224,0.021263,0.019858,0.026744
1,1,0.005612,0.005995,0.007382,0.005180,0.007730
2,2,0.013286,0.010998,0.012470,0.010906,0.013163
3,3,0.037278,0.034350,0.036937,0.036076,0.033870
4,4,0.050503,0.048746,0.046275,0.048314,0.047768
5,5,0.007538,0.006291,0.007801,0.006257,0.005641
6,6,0.006517,0.006845,0.009251,0.007541,0.006055
7,7,0.065181,0.050586,0.052449,0.051802,0.055311
8,8,0.013044,0.015069,0.013926,0.013104,0.012576
9,9,0.013535,0.014616,0.013846,0.011580,0.009947


In [183]:
best_features_pi_fs = pi_fs.get_selected_features(threshold=0.99)
len(best_features_pi_fs)

21

In [184]:
best_features_pi_fs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 40]

In [185]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_pi_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9604289764385765

# Жадные методы отбора

In [193]:
class SequentialForwardFeatureSelection:
    def __init__(self, model, n_folds, k, stratified=True, scoring='roc_auc'):
        self.model = model
        self.n_folds = n_folds
        self.k = k
        self.stratified = True
        self.scoring = scoring

    def _find_feature_to_add(self, x, y, cur_subset, set_to_choose_from):
        best_score = float('-inf')
        best_feature = None

        print('*'*30)
        print('Looking for feature to ADD...')

        for feature in set_to_choose_from:
            cur_subset_copy = cur_subset.copy()
            cur_subset_copy += [feature]

            subset_x = x[:, cur_subset_copy]
            if self.stratified:
                cv = StratifiedKFold(n_splits=self.n_folds, shuffle=True)
            else:
                cv = KFold(n_splits=self.n_folds, shuffle=True)
            score = cross_val_score(estimator=self.model, X=subset_x, y=y, cv=cv, scoring=self.scoring).mean()
            if score > best_score:
                best_feature = feature
                best_score = score
                
        print(f'Best score = {best_score:.4f}')
        return best_feature

    def _find_feature_to_delete(self, x, y, cur_subset):
        if self.stratified:
            cv = StratifiedKFold(n_splits=self.n_folds, shuffle=True)
        else:
            cv = KFold(n_splits=self.n_folds, shuffle=True)
        best_score = cross_val_score(estimator=self.model, X=x[:, cur_subset], y=y, cv=cv, scoring=self.scoring).mean()
        worst_feature = None

        print('='*30)
        print('Looking for feature to DELETE...')
        print(f'Best score = {best_score:.4f}')

        for feature in cur_subset:
            cur_subset_copy = cur_subset.copy()
            del cur_subset_copy[cur_subset_copy.index(feature)]

            subset_x = x[:, cur_subset_copy]
            if self.stratified:
                cv = StratifiedKFold(n_splits=self.n_folds, shuffle=True)
            else:
                cv = KFold(n_splits=self.n_folds, shuffle=True)
            score = cross_val_score(estimator=self.model, X=subset_x, y=y, cv=cv, scoring=self.scoring).mean()
            if score > best_score:
                worst_feature = feature
                best_score = score
        return worst_feature

    def fit(self, x, y):
        best_features = []
        set_to_choose_from = list(range(x.shape[1]))

        best_feature = self._find_feature_to_add(x=x, y=y, cur_subset=best_features, set_to_choose_from=set_to_choose_from)
        best_features += [best_feature]
        print(f'Added feature {best_feature}, current subset = {best_features}')
        del set_to_choose_from[set_to_choose_from.index(best_feature)]

        while len(best_features) < self.k:
            best_feature = self._find_feature_to_add(x=x, y=y, cur_subset=best_features, set_to_choose_from=set_to_choose_from)
            best_features += [best_feature]
            print(f'Added feature {best_feature}, current subset = {best_features}')
            del set_to_choose_from[set_to_choose_from.index(best_feature)]

            if len(best_features) > 2:
                worst_feature = self._find_feature_to_delete(x=x, y=y, cur_subset=best_features)
                if worst_feature == best_feature:
                    break

                if worst_feature is not None:
                    del best_features[best_features.index(worst_feature)]
                    print(f'Deleted feature {worst_feature}, current subset = {best_features}')
                    set_to_choose_from += [worst_feature]

                    while len(best_features) > 2:
                        worst_feature = self._find_feature_to_delete(x=x, y=y, cur_subset=best_features)
                        if worst_feature is None:
                            break
                        del best_features[best_features.index(worst_feature)]
                        print(f'Deleted feature {worst_feature}, current subset = {best_features}')
                        set_to_choose_from += [worst_feature]
        return best_features

In [194]:
# самый долгий

%%time
seq_fs = SequentialForwardFeatureSelection(
    model=LGBMClassifier(max_depth=5, n_estimators=500, learning_rate=0.05, verbose=-100),
    n_folds=5,
    k=50)
best_features_seq_fs = seq_fs.fit(x, y)
len(best_features_seq_fs)

******************************
Looking for feature to ADD...
Best score = 0.6613
Added feature 7, current subset = [7]
******************************
Looking for feature to ADD...
Best score = 0.7168
Added feature 10, current subset = [7, 10]
******************************
Looking for feature to ADD...
Best score = 0.7557
Added feature 22, current subset = [7, 10, 22]
Looking for feature to DELETE...
Best score = 0.7526
******************************
Looking for feature to ADD...
Best score = 0.7961
Added feature 24, current subset = [7, 10, 22, 24]
Looking for feature to DELETE...
Best score = 0.7964
******************************
Looking for feature to ADD...
Best score = 0.8342
Added feature 13, current subset = [7, 10, 22, 24, 13]
Looking for feature to DELETE...
Best score = 0.8325
******************************
Looking for feature to ADD...
Best score = 0.8685
Added feature 3, current subset = [7, 10, 22, 24, 13, 3]
Looking for feature to DELETE...
Best score = 0.8677
***********

17

In [195]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_seq_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9579893735297735

In [197]:
set(best_features_lgm_fs) & set(best_features_shap_fs), set(best_features_lgm_fs) & set(best_features_pi_fs), set(best_features_pi_fs) & set(best_features_shap_fs)

({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 40})

# Но что если признаков очень много?

In [196]:
x, y = make_classification(
    n_samples=100000, n_features=1000, n_informative=35, n_redundant=7, n_repeated=8, n_clusters_per_class=4, shift=0.8, scale=3.0, shuffle=False)

In [199]:
%%time
lgm_fs = LGMFeatureSelection()
lgm_fs.fit(x, y, importance_type='split')

CPU times: total: 18min 55s
Wall time: 3min 18s


In [200]:
# замерим метрику на кросс валидации
cross_val_score(LGBMClassifier(verbose=-100), x, y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9699239396992393

In [201]:
lgm_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,360,358,349,396,354
1,1,370,378,361,366,367
2,2,246,245,255,260,288
3,3,306,299,305,293,313
4,4,283,314,294,298,302
5,5,401,398,375,376,396
6,6,269,259,266,263,251
7,7,319,326,333,313,352
8,8,257,269,279,249,286
9,9,276,257,250,261,273


In [207]:
best_features_lgm_fs = lgm_fs.get_selected_features(threshold=0.85)
len(best_features_lgm_fs)

42

In [208]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_lgm_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9699724176997242

In [204]:
# видимо имеет смысл осторожнее выбирать отсечку, либо объединять результаты нескольких методов